## install bpy
- [參考網站](https://www.zhihu.com/question/386776864)
- steps
    1. `conda create -n ***`
    2. 至 https://pypi.org/project/bpy/#files 查看 python 版本 (cp***)
    3. 安裝對應 python 版本 `conda install python=***`
    4. `pip install bpy`
- (記得切換 kernel -> `conda activate bpy`)
## steps and references
- preprocessing
    - [convert curve to mesh](https://blender.stackexchange.com/questions/265215/how-can-i-convert-a-curve-to-a-mesh-object)
    - [get vertices from mesh](https://blender.stackexchange.com/questions/1311/how-can-i-get-vertex-positions-from-a-mesh)
    - [get original colors](https://blenderartists.org/t/svg-import-remove-redundant-materials/693325/4)
- create mesh
    - [triangulate polygon in shapely](https://stackoverflow.com/questions/65019170/how-do-you-triangulate-a-polygon-in-shapely)
- simplification
    - ~~[tripy](https://github.com/linuxlewis/tripy)~~
    - ~~[optimization](https://github.com/meshpro/optimesh)~~
    - [triangulate](https://github.com/lionfish0/earclip)
## other packages
- `pip install scipy`
- `pip install shapely`
- `pip install geopandas`
- `pip install geovoronoi`
- `pip install tripy`
- `pip install optimesh`

In [1]:
import os
import warnings

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import bpy
import matplotlib.pyplot as plt
import shapely.wkt
import geopandas as gpd
import tripy
import optimesh
import torch
import colorsys

from shapely.geometry import Polygon
from shapely.ops import triangulate
from matplotlib.lines import Line2D
from geovoronoi import voronoi_regions_from_coords
from quad_mesh_simplify import simplify_mesh
from collections import OrderedDict
from torch_geometric.data import Data

In [26]:
# import file
file_path = "./datasets/svg/011-library.svg"
# file_path = "./datasets/svg/037-time.svg"
# file_path = "./datasets/svg/027-diamond.svg"
# file_path = "./datasets/svg/019-watermelon.svg"
# file_path = "./datasets/svg/024-book.svg"
# file_path = "./datasets/svg/032-firewood.svg"
# file_path = "./datasets/svg/050-shopping cart.svg"

# clean the scene
bpy.ops.object.select_all()
bpy.ops.object.delete()
bpy.ops.import_curve.svg(filepath=file_path)

{'FINISHED'}

In [27]:
# convert curve to mesh
idx = 0
for ob in bpy.data.objects:
    if ob.type == "CURVE":
        mesh = bpy.data.meshes.new_from_object(ob)
        new_obj = bpy.data.objects.new("mesh_obj" + str(idx), mesh)
        new_obj.matrix_world = ob.matrix_world
        bpy.context.collection.objects.link(new_obj)
        idx += 1

In [28]:
# get all curve meshes
colors = {}
nodes = {}
idx = 0
hsv = {}
for ob in bpy.data.objects:
    if ob.type == "MESH" and "mesh_obj" in ob.name:
        try:
            # get mesh
            rgb = ob.material_slots[0].material.diffuse_color
            colors[idx] = np.array([rgb[0], rgb[1], rgb[2]])
            hsv[idx] = np.array(colorsys.rgb_to_hsv(rgb[0], rgb[1], rgb[2]))
            
            # get vertices
            v = ob.data.vertices[0]
            coords = [(ob.matrix_world @ v.co) for v in ob.data.vertices]  # (x, y, z)
            nodes[idx] = []
            for x, y, z in coords:
                nodes[idx].append([x, y]) 
            nodes[idx] = np.array(nodes[idx])
            
            idx += 1
        except:
            continue

In [32]:
# triangulate
from shapely.ops import triangulate

polys = {}
all_polys = []
for i, node in nodes.items():
    polys[i] = []
    all_polys.append(None)
    if len(node) < 4: continue
    
    poly = Polygon(node).buffer(0.001)
    all_polys[i] = poly
    tri = triangulate(poly)
    for po in tri:
        xx, yy = po.exterior.coords.xy
        temp = []
        for x, y in zip(xx, yy):
            temp.append([x, y])
        polys[i].append(temp[:3])
    polys[i] = np.array(polys[i])

In [34]:
# for first stage
all_points = []
all_edges = []
all_rgb = []
all_hsv = []
all_cluster = []
add_num = 0
new_polys = []
idx = 0

for i, poly in polys.items():
    if len(poly) == 0: continue
    new_polys.append(all_polys[i])
    
    p_i = 0
    pos2idx = {}
    points = []
    cells = []
    for n1, n2, n3 in poly:
        if tuple(n1) not in pos2idx:
            pos2idx[tuple(n1)] = p_i
            points.append(n1)
            all_rgb += [colors[i].tolist()]
            all_hsv += [hsv[i].tolist()]
            p_i += 1
        if tuple(n2) not in pos2idx:
            pos2idx[tuple(n2)] = p_i
            points.append(n2)
            all_rgb += [colors[i].tolist()]
            all_hsv += [hsv[i].tolist()]
            p_i += 1
        if tuple(n3) not in pos2idx:
            pos2idx[tuple(n3)] = p_i
            points.append(n3)
            all_rgb += [colors[i].tolist()]
            all_hsv += [hsv[i].tolist()]
            p_i += 1       
        cells.append([pos2idx[tuple(n1)], pos2idx[tuple(n2)], pos2idx[tuple(n3)]])
        
    points = np.array(points)
    cells = np.array(cells) 
    points, cells = optimesh.optimize_points_cells(
        points, cells, "cpt-quasi-newton", 1.0e-5, 100
    )
    
    if idx == 0:
        all_points = points
    else:
        all_points = np.concatenate((all_points, points), axis=0)
    for a, b, c in cells:
        all_edges.append([a+add_num, b+add_num])
        all_edges.append([b+add_num, c+add_num])
        all_edges.append([c+add_num, a+add_num])
    add_num += len(points)
    
    all_cluster += [idx for _ in range(len(points))]
    idx += 1
    
all_points = np.array(all_points)
all_edges = np.array(all_edges)
all_rgb = np.array(all_rgb)
all_hsv = np.array(all_hsv)
all_cluster = np.array(all_cluster)

In [35]:
print(f"number of all points: {len(all_points)}")
print(f"number of all edges: {len(all_edges)}")
print(f"rgb: {all_rgb.shape}; hsv: {all_hsv.shape}; cluster: {all_cluster.shape}")

number of all points: 3752
number of all edges: 15780
rgb: (3752, 3); hsv: (3752, 3); cluster: (3752,)


In [36]:
# check overlap
def block_intersect(i, j, poly1, poly2, group_edges, group_edges_type):
    a1 = poly1.area
    a2 = poly2.area
    intersect = poly1.intersection(poly2).area
    
    if abs(intersect - min(a1, a2)) < 0.000001:  # in
        group_edges.append([i, j])
        group_edges.append([j, i])
        group_edges_type.append(2)
        group_edges_type.append(2)
    elif intersect != 0:  # overlap
        group_edges.append([i, j])
        group_edges.append([j, i])
        group_edges_type.append(1)
        group_edges_type.append(1)
    else:  # adjacent
        group_edges.append([i, j])
        group_edges.append([j, i])
        group_edges_type.append(0)
        group_edges_type.append(0)
        
    return group_edges, group_edges_type

# check group connection
group_edges = []
group_edges_type = []

for i in range(len(new_polys)-1):
    for j in range(i+1, len(new_polys)):
        group_edges, group_edges_type = block_intersect(i, j, new_polys[i], new_polys[j], group_edges, group_edges_type)

In [10]:
# create pyg data
# nodes
x = torch.Tensor(all_points)
y_rgb = torch.Tensor(all_rgb)
y_hsv = torch.Tensor(all_hsv)
y_cluster = torch.Tensor(all_cluster)

# edges
m = all_edges.shape[0]
edges = np.zeros([2*m, 2]).astype(np.int64)
edge_attr = np.zeros([2*m, 4]).astype(np.float32)
for e, (s,t) in enumerate(all_edges):
    edges[e, 0] = s
    edges[e, 1] = t
    edges[m+e, 0] = t
    edges[m+e, 1] = s
    
    edge_attr[e, :2] = all_points[s]
    edge_attr[e, 2:] = all_points[t]
    edge_attr[m+e, :2] = all_points[t]
    edge_attr[m+e, 2:] = all_points[s]
edges = torch.Tensor(np.transpose(edges)).type(torch.long)
edge_attr = torch.Tensor(edge_attr)

group_edges = torch.Tensor(np.transpose(group_edges)).type(torch.long)
group_edge_attr = torch.Tensor(group_edges_type)

data = Data(x=x, edge_index=edges, rgb=y_rgb, hsv=y_hsv, edge_attr=edge_attr, cluster=y_cluster, group_edge_index=group_edges, group_edge_attr=group_edge_attr)
print(data)

Data(x=[835, 2], edge_index=[2, 6594], edge_attr=[6594, 4], rgb=[835, 3], hsv=[835, 3], cluster=[835], group_edge_index=[2, 30], group_edge_attr=[30])


---
---
## 2-stage model

In [11]:
import torch
import torch.optim as optim
import torch.nn.functional as F

from tqdm import tqdm
from torch import nn, Tensor
from torch_scatter import scatter
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing, GCNConv, global_mean_pool
from torch_geometric.utils import add_self_loops, degree

In [14]:
# hyperparameters
torch.manual_seed(16)

batch_size = 1
num_features = 2
num_output = 3
num_epoch = 50

# _train = int(len(dataset) * 0.9)
# _val = _train + int(len(dataset) * 0.05)
# _test = len(dataset) - _val

# # create dataloader
# train_set, val_set, test_set = dataset[:_train], dataset[_train:_val], dataset[_val:]
# train_svg, val_svg, test_svg = imgs[:_train], imgs[_train:_val], imgs[_val:]
# train_png, val_png, test_png = png[:_train], png[_train:_val], png[_val:]

# train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
# test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# print(f"Training Data: {len(train_set)}\nValidation Data: {len(val_set)}\nTesting Data: {len(test_set)}")

In [23]:
class GraphEncoder(nn.Module):
    def __init__(self):
        super(GraphEncoder, self).__init__()
        self.conv1 = GCNConv(2, 16, improved=True)
        self.conv2 = GCNConv(16, 64, improved=True)
        
    def forward(self, x, edge_index, cluster):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        
        x = scatter(x, cluster, dim=0, reduce='mean')
        return x

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(64, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Linear(16, 3),
            nn.BatchNorm1d(3),
            nn.ReLU(),
        )
        
    def forward(self, x):
        return self.layers(x)
    
# class SVGConv(MessagePassing):
#     def __init__(self,
#                  in_channels: int,
#                  out_channels: int,
#                  improved: bool = True,
#                  add_self_loops: bool = True,
#                  normalize: bool = True,
#                  bias: bool = True,
#                  **kwargs
#     ):
#         super(SVGConv, self).__init__(aggr="mean", **kwargs)
#         self.in_channels = in_channels
#         self.out_channels = out_channels
#         self.improved = improved
#         self.add_self_loops = add_self_loops
#         self.normalize = normalize
        
#         self.lin = nn.Linear(in_channels, out_channels, bias=False)
        
#         if bias:
#             self.bias = nn.Parameter(torch.Tensor(out_channels))
#         else:
#             self.register_parameter("bias", None)
        
#         self.reset_parameters()
        
#     def reset_parameters(self):
#         torch.nn.init.uniform_(self.lin.weight)
#         if self.bias is not None:
#             torch.nn.init.normal_(self.bias, mean=0.0, std=0.1)
        
#     def forward(self, x, edge_index, edge_attr):
#         out = self.propagate(edge_index, x=x, edge_attr=edge_attr)
#         return out
    
#     def message(self, x_i, x_j, edge_attr):
#         if edge_attr == 2:  # overlap
            
#         elif edge_attr == 3:  # contain
#         else:  # adjacent
            
#         return super().message(x_j)
    
class GraphNet(nn.Module):
    def __init__(self):
        super(GraphNet, self).__init__()
        self.encode = GraphEncoder()
        self.gcn1 = GCNConv(64, 128)
        self.gcn2 = GCNConv(128, 64)
        # self.gcn1 = SVGConv(64, 128)
        # self.gcn2 = SVGConv(128, 64)
        self.fc = MLP()
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        cluster = data.cluster.type(torch.long)
        group_edge_index, group_edge_attr = data.group_edge_index, data.group_edge_attr
        
        feature = self.encode(x, edge_index, cluster)
        
        out = self.gcn1(feature, group_edge_index, group_edge_attr)
        out = F.relu(out)
        out = self.gcn2(out, group_edge_index, group_edge_attr)
        out = F.relu(out)
        out = self.fc(out)
        
        return out

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.MSELoss(reduction='mean')

In [25]:
# training
train_losses = []
val_losses = []
best_loss = float('inf')
err = 0

for epoch in range(2):  # num_epoch
    train_loss = 0
    val_loss = 0
    
    model.train()
    
    data = data.to(device)
    if data.x.shape[0] == 0:
        err += 1
        continue
    
    optimizer.zero_grad()
    out = model(data)
    
    cluster = data.cluster.type(torch.long)
    rgb = scatter(data.rgb, cluster, dim=0, reduce='mean')
    loss = criterion(out, rgb)
    
    print(out)
    print(rgb)
    print(loss.item())
    
    loss.backward()
    optimizer.step()
    train_loss += loss.item()

tensor([[0.0000, 0.7067, 0.0000],
        [0.0000, 0.7067, 0.0000],
        [0.0000, 0.7067, 0.0000],
        [1.4067, 0.0000, 1.4124],
        [1.4067, 0.0000, 1.4124],
        [0.0000, 0.7067, 0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.3185, 0.4910, 1.0000],
        [0.1301, 0.3185, 0.9911],
        [0.8228, 0.7913, 0.8963],
        [0.9823, 0.0612, 0.1144],
        [0.9823, 0.1500, 0.1144],
        [1.0000, 0.9047, 0.9216]], device='cuda:0')
0.5238862633705139
tensor([[0.6899, 0.0000, 0.6897],
        [0.6899, 0.0000, 0.6897],
        [0.6899, 0.0000, 0.6897],
        [0.0000, 1.3851, 0.0000],
        [0.0000, 1.3851, 0.0000],
        [0.6899, 0.0000, 0.6897]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor([[0.3185, 0.4910, 1.0000],
        [0.1301, 0.3185, 0.9911],
        [0.8228, 0.7913, 0.8963],
        [0.9823, 0.0612, 0.1144],
        [0.9823, 0.1500, 0.1144],
        [1.0000, 0.9047, 0.9216]], device='cuda:0')
0.4372161030769348


In [ ]:
# # training
# train_losses = []
# val_losses = []
# best_loss = float('inf')
# err = 0

# for epoch in range(2):  # num_epoch
#     train_loss = 0
#     val_loss = 0
    
#     model.train()
#     for i, data in enumerate(tqdm(train_loader)):
#         data = data.to(device)
#         if data.x.shape[0] == 0:
#             err += 1
#             continue
        
#         optimizer.zero_grad()
#         out = model(data)
#         # h = data.h.type(torch.LongTensor).to(device)
#         # loss = criterion(out, h)
#         loss = criterion(out, data.rgb)
        
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()
        
#     model.eval()
#     for i, data in enumerate(tqdm(val_loader)):
#         data = data.to(device)
#         out = model(data)
        
#         # h = data.h.type(torch.LongTensor).to(device)
#         # loss = criterion(out, h)
#         loss = criterion(out, data.rgb)
        
#         val_loss += loss.item()
    
#     train_avg = train_loss / len(train_loader)
#     val_avg = val_loss / len(val_loader)
#     train_losses.append(loss.item())
#     val_losses.append(loss.item())
    
#     print(f'Epoch {epoch}\tTraining Loss: {train_avg}\tValidation Loss: {val_avg}')
    
#     if val_avg < best_loss:
#         print(f'Validation Loss Decreased({best_loss:.6f}--->{val_avg:.6f})\tSaving The Model')
#         best_loss = val_avg
#         torch.save(model.state_dict(), 'best_checkpoint.pth')